# Using `vl_datasets` with `torchvision` and `torch`
This notebook shows how to load clean datasets from `vl-datasets` into [torchvision](https://github.com/pytorch/vision) for training. 

<!--<badge>--><a href="https://colab.research.google.com/github/visual-layer/vl-datasets/blob/main/notebooks/train-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

## Installation & Setting Up

In [ ]:
!pip install vl-datasets

## Define the preprocessing transforms
We define the preprocessing transforms for the dataset. We have two transforms: `train_transform` and `valid_transform`.

In [2]:
import torchvision.transforms as transforms

train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

## Download food-101 Dataset

The easiest way to get the clean version of food-101 dataset is to use `vl_dataset` package.

In [4]:
from vl_datasets import CleanFood101

train_dataset = CleanFood101('./', split='train', exclude_csv='csv_files/food_101_vl-datasets_analysis.csv', transform=train_transform)
valid_dataset = CleanFood101('./', split='test', exclude_csv='csv_files/food_101_vl-datasets_analysis.csv', transform=valid_transform)

Using downloaded and verified file: ./food-101.tar.gz
Extracting ./food-101.tar.gz to ./
Excluded 407 from the train set
Excluded 91 from the test set


Upon completion, you'd have with a `food-101` folder in your local directory.

```
food-101/
├── images
│   ├── apple_pie
│   │   ├── 0001.jpg
│   │   └── 0002.jpg
│   ├── baby_back_ribs
│   ├── baklava
│   ├── ...
│   ├── ...
│   ├── ...
│   └── waffles
├── meta
|   ├── classes.txt
│   ├── labels.txt
│   ├── test.json
│   ├── test.txt
│   ├── train.json
│   └── train.txt
├── license_agreement.txt
└── README.txt
```


If you'd like to download the food-101 manually, you run the following codes to download and extract the folder into your local directory.

```shell
wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
tar -xf food-101.tar.gz
```

Next, load the directory into a `CleanFood101` class.

```python
train_dataset = CleanFood101('./', download=False, split='train', exclude_csv='csv_files/food_101_vl-datasets_analysis.csv', transform=train_transform)
valid_dataset = CleanFood101('./', download=False, split='test', exclude_csv='csv_files/food_101_vl-datasets_analysis.csv', transform=train_transform)
```

View the problematic images that are listed in the `.csv` file.

In [5]:
print(train_dataset.excluded_files[:5])

['churros/1440917.jpg', 'churros/1944265.jpg', 'churros/3867252.jpg', 'churros/644700.jpg', 'hot_and_sour_soup/1674538.jpg']


## Import `torch`

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision

We can also view the exclude files with:

In [7]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True, pin_memory=True)

## Define the model architecture
Let's construct a basic convolutional model, Resnet18 from Torchvision.

In [8]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

## Define the loss function and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train the model
Now, let's write a simple training loop to train the model for 10 epochs on a GPU or CPU.

In [10]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")


Using device: cuda
Epoch 1 - Loss: 2.3851431931479503
Epoch 2 - Loss: 1.8027292704178115
Epoch 3 - Loss: 1.608138797646862
Epoch 4 - Loss: 1.4756208367266896
Epoch 5 - Loss: 1.386723577370078


## Evaluate the model
Finally we evaluate the model on the validation set and prints it's accuracy.

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")


Accuracy: 73.34154775627012
